In [1]:
import pandas as pd

In [27]:
def create_df(cols, index, key_col=True):
    data = {'key': ['k' + str(i) for i in index]}
    col_data = {c: [str(c) + str(i) for i in index] 
            for c in cols}
    if key_col:
        data.update(col_data)
        return pd.DataFrame(data)
    
    return pd.DataFrame(col_data)

In [28]:
create_df(list('ABC'), range(0, 3))

,key,A,B,C
0,k0,A0,B0,C0
1,k1,A1,B1,C1
2,k2,A2,B2,C2


In [51]:
create_df(list('BCD'), range(2, 4), key_col=False)

,B,C,D
0,B2,C2,D2
1,B3,C3,D3


## One-to-one (1:1)

In [52]:
df1 = create_df(list('AB'), range(3))
df1

,key,A,B
0,k0,A0,B0
1,k1,A1,B1
2,k2,A2,B2


In [53]:
df2 = create_df(list('CD'), range(3))
df2

,key,C,D
0,k0,C0,D0
1,k1,C1,D1
2,k2,C2,D2


In [54]:
pd.merge(df1, df2)

,key,A,B,C,D
0,k0,A0,B0,C0,D0
1,k1,A1,B1,C1,D1
2,k2,A2,B2,C2,D2


## Many-to-one (m:1)

In [75]:
df3 = create_df(list('AB'), range(4), key_col=False)
df3.loc[3, 'B'] = 'B2'
df3

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B2


In [76]:
df4 = create_df(list('BC'), range(3), key_col=False)
df4

,B,C
0,B0,C0
1,B1,C1
2,B2,C2


In [77]:
pd.merge(df3, df4)

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2
3,A3,B2,C2


## Many-to-Many (m:m)

In [78]:
df5 = df4.append({'B': 'B1', 'C': 'C3'}, ignore_index=True)
df5

,B,C
0,B0,C0
1,B1,C1
2,B2,C2
3,B1,C3


In [79]:
pd.merge(df3, df5)

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A1,B1,C3
3,A2,B2,C2
4,A3,B2,C2


## Relational Algebra

In [30]:
df_left = create_df(list('AB'), range(4))
df_left

,key,A,B
0,k0,A0,B0
1,k1,A1,B1
2,k2,A2,B2
3,k3,A3,B3


In [25]:
df_right = create_df(list('CD'), range(4))
df_right

,C,D,key
0,C0,D0,k0
1,C1,D1,k1
2,C2,D2,k2
3,C3,D3,k3


In [31]:
pd.merge(df_left, df_right, on='key')

,key,A,B,C,D
0,k0,A0,B0,C0,D0
1,k1,A1,B1,C1,D1
2,k2,A2,B2,C2,D2
3,k3,A3,B3,C3,D3


In [32]:
df_right = create_df('CD', range(1, 5))
df_right

,key,C,D
0,k1,C1,D1
1,k2,C2,D2
2,k3,C3,D3
3,k4,C4,D4


In [33]:
pd.merge(df_left, df_right, on='key')

,key,A,B,C,D
0,k1,A1,B1,C1,D1
1,k2,A2,B2,C2,D2
2,k3,A3,B3,C3,D3


In [34]:
pd.merge(df_left, df_right, on='key', how='outer')

,key,A,B,C,D
0,k0,A0,B0,NaN,NaN
1,k1,A1,B1,C1,D1
2,k2,A2,B2,C2,D2
3,k3,A3,B3,C3,D3
4,k4,NaN,NaN,C4,D4


In [35]:
# left outer
pd.merge(df_left, df_right, on='key', how='left') 

,key,A,B,C,D
0,k0,A0,B0,NaN,NaN
1,k1,A1,B1,C1,D1
2,k2,A2,B2,C2,D2
3,k3,A3,B3,C3,D3


In [36]:
# right outer
pd.merge(df_left, df_right, on='key', how='right') 

,key,A,B,C,D
0,k1,A1,B1,C1,D1
1,k2,A2,B2,C2,D2
2,k3,A3,B3,C3,D3
3,k4,NaN,NaN,C4,D4


## Join on index

In [39]:
df_left = create_df(list('AB'), range(3)).set_index('key')
df_left

,A,B
key,,
k0,A0,B0
k1,A1,B1
k2,A2,B2


In [40]:
df_right = create_df(list('CD'), range(1, 4)).set_index('key')
df_right

,C,D
key,,
k1,C1,D1
k2,C2,D2
k3,C3,D3


In [41]:
df_left.join(df_right, how='inner')

,A,B,C,D
key,,,,
k1,A1,B1,C1,D1
k2,A2,B2,C2,D2


In [42]:
df_left.join(df_right, how='outer')

,A,B,C,D
key,,,,
k0,A0,B0,NaN,NaN
k1,A1,B1,C1,D1
k2,A2,B2,C2,D2
k3,NaN,NaN,C3,D3


Same behavior using `pd.merge`:

In [43]:
pd.merge(df_left, df_right,
         left_index=True, right_index=True, how='inner')

,A,B,C,D
key,,,,
k1,A1,B1,C1,D1
k2,A2,B2,C2,D2


In [44]:
pd.merge(df_left, df_right,
         left_index=True, right_index=True, how='outer')

,A,B,C,D
key,,,,
k0,A0,B0,NaN,NaN
k1,A1,B1,C1,D1
k2,A2,B2,C2,D2
k3,NaN,NaN,C3,D3


## Join key columns on an index

Two completely equivalent function:

- `left.join(right, on=key_or_keys)`

- `pd.merge(left, right, left_on=key_or_keys, right_index=True, how='left', sort=False)`

In [105]:
df_left = create_df(list('AB'), range(3))
df_left

,key,A,B
0,k0,A0,B0
1,k1,A1,B1
2,k2,A2,B2


In [106]:
df_right = create_df(list('CD'), range(1, 4)).set_index('key')
df_right

,C,D
key,,
k1,C1,D1
k2,C2,D2
k3,C3,D3


In [107]:
df_left.join(df_right, on='key')

,key,A,B,C,D
0,k0,A0,B0,NaN,NaN
1,k1,A1,B1,C1,D1
2,k2,A2,B2,C2,D2


Same behavior with `pd.merge()`

In [109]:
pd.merge(df_left, df_right, 
         left_on='key', right_index=True, how='left')

,key,A,B,C,D
0,k0,A0,B0,NaN,NaN
1,k1,A1,B1,C1,D1
2,k2,A2,B2,C2,D2


In [111]:
df_left

,key,A,B
0,k0,A0,B0
1,k1,A1,B1
2,k2,A2,B2


In [112]:
df_right

,C,D
key,,
k1,C1,D1
k2,C2,D2
k3,C3,D3


In [113]:
pd.merge(df_left.set_index('key'), df_right, 
         left_index=True, right_index=True, how='left')

,A,B,C,D
key,,,,
k0,A0,B0,NaN,NaN
k1,A1,B1,C1,D1
k2,A2,B2,C2,D2


In [114]:
df_right.reset_index(level=0, inplace=True)
df_right

,key,C,D
0,k1,C1,D1
1,k2,C2,D2
2,k3,C3,D3


In [117]:
df_left

,A,B
key,,
k0,A0,B0
k1,A1,B1
k2,A2,B2


In [118]:
pd.merge(df_left, df_right, left_index=True, right_on='key')

,A,B,key,C,D
0,A1,B1,k1,C1,D1
1,A2,B2,k2,C2,D2


## `left_on` and `right_on`

In [92]:
df_left = create_df(list('AB'), range(4), key_col=False)
df_left.rename(columns={'B': 'B_Left'}, inplace=True)
df_left

,A,B_Left
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [93]:
df_right = create_df(list('BC'), range(1, 5), key_col=False)
df_right.rename(columns={'B': 'B_Right'}, inplace=True)
df_right

,B_Right,C
0,B1,C1
1,B2,C2
2,B3,C3
3,B4,C4


In [95]:
pd.merge(df_left, df_right, 
         left_on='B_Left', right_on='B_Right',
         how='outer')

,A,B_Left,B_Right,C
0,A0,B0,NaN,NaN
1,A1,B1,B1,C1
2,A2,B2,B2,C2
3,A3,B3,B3,C3
4,NaN,NaN,B4,C4


In [120]:
df_left = create_df(list('AB'), range(3))
df_left

,key,A,B
0,k0,A0,B0
1,k1,A1,B1
2,k2,A2,B2


In [122]:
df_right = create_df(list('BC'), range(1, 4))
df_right

,key,B,C
0,k1,B1,C1
1,k2,B2,C2
2,k3,B3,C3


In [123]:
pd.merge(df_left, df_right)

,key,A,B,C
0,k1,A1,B1,C1
1,k2,A2,B2,C2


In [129]:
pd.merge(df_left, df_right, on='B', suffixes=['_L', '_R'])

,key_L,A,B,key_R,C
0,k1,A1,B1,k1,C1
1,k2,A2,B2,k2,C2


In [130]:
pd.merge(df_left, df_right, left_on='B', right_on='B', suffixes=['_L', '_R'])

,key_L,A,B,key_R,C
0,k1,A1,B1,k1,C1
1,k2,A2,B2,k2,C2
